In [5]:
import tensorflow as tf
import numpy as np
import math
import fce_api as fd
import re
import sklearn as sk
import sklearn.metrics as skm

# variables
# data
data = fd.extract_data('fce_train.gold.max.rasp.old_cat.m2')

window_size = 3

labels = []
PAD = 0
display_step = 1

# classes
has_error = 1
no_error = 0

# learning variables
learning_rate = 0.0001
epochs = 44
batch_size = 2000


# generating the word windows, including the spaces.
def feed_windows_with_spaces(_data, _window_size):
    windows = []
    for sentence, errors in _data:
        tokens = re.split(r'(\s+)', sentence)
        word_window_size = min(len(tokens), _window_size)
        for i in range(0, len(tokens) - word_window_size + 1):
            window_tuple = (tokens[i:i + word_window_size], )
            window_range = range(round(i / 2), round((i + word_window_size) / 2))
            for error in errors:
                if error[0] in window_range or error[1] in window_range:
                    if len(window_tuple) < 2:
                        window_tuple = window_tuple + (has_error, )
            if len(window_tuple) == 1:
                window_tuple = window_tuple + (no_error, )
            windows.append(window_tuple)
    return windows


# generating the word windows only with the tokens, excluding the spaces.
def feed_windows_only_tokens(_data, _window_size):
    windows = []
    for sentence, errors in _data:
        tokens = sentence.split()
        word_window_size = min(len(tokens), _window_size)
        for i in range(0, len(tokens) - word_window_size + 1):
            window_tuple = (tokens[i:i + word_window_size], )
            window_range = range(i, i + word_window_size)
            for error in errors:
                if error[0] in window_range or error[1] in window_range:
                    if len(window_tuple) < 2:
                        window_tuple = window_tuple + (has_error, )
            if len(window_tuple) == 1:
                window_tuple = window_tuple + (no_error, )
            windows.append(window_tuple)
    return windows

feed_windows = feed_windows_only_tokens(data, window_size)
# feed_windows = feed_windows[:200]
# changing the training data
print('feed_windows: ', '%d' % len(feed_windows), 'sentences: ', '%d' % len(data))

# feed_windows = feed_windows[:42000]

print('ready')       

vocab = {'<PAD>': PAD}
feed_windows_np = np.full([len(feed_windows), window_size], PAD)
labels = np.full([len(feed_windows), 2], 2)

for i, window in enumerate(feed_windows):
    window_length = len(window[0]) 
    seq = np.full(window_length, 0)
    for j, token in enumerate(window[0]):
        if token not in vocab:
            vocab[token] = len(vocab)
        seq[j] = vocab[token]  
    feed_windows_np[i, 0:window_length] = seq
    if window[1] == has_error:
        labels[i] = [0, 1]
    else:
        labels[i] = [1, 0]

x = tf.placeholder(tf.float32, [None, window_size]) # data with the size of the window currently set to 6
y = tf.placeholder(tf.float32, [None, 2])

W = tf.Variable(tf.zeros([window_size, 2]))
b = tf.Variable(tf.zeros([2]))

# Construct model
model = tf.matmul(x, W) # Sigmoid

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(model, y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


predict = tf.argmax(model, 1)

print(len(feed_windows_np))

# Splitting to train and test
train = feed_windows_np[:len(feed_windows_np)//2 - ((len(feed_windows_np)//2) % batch_size)]
train_labels = labels[:len(feed_windows_np)//2 - ((len(feed_windows_np)//2) % batch_size)]
test = feed_windows_np[(len(feed_windows_np) * 2) // 3:]
test_labels = labels[(len(feed_windows_np) * 2) // 3:]
total_batches = int(math.floor(len(train) / batch_size))

avg_costs = []

with tf.Session() as sess:
    # Initializing the variables
    init = tf.initialize_all_variables()
    sess.run(init)
    y_true = test_labels

    # Training cycle
    for epoch in range(epochs):
        for start, end in zip(range(0, len(train), 128), range(128, len(train) + 1, 128)):
            sess.run(optimizer, feed_dict={x: train[start:end], y: train_labels[start:end]})
        y_pred = sess.run(predict, feed_dict={x: test})
        print(epoch, np.mean(y_true == y_pred))
        print("Precision", skm.precision_score(y_true, y_pred))
        print("Recall", skm.recall_score(y_true, y_pred))
        print("f1_score", skm.f1_score(y_true, y_pred))
        # print("confusion_matrix")
        # print(sk.metrics.confusion_matrix(y_true, y_pred))



feed_windows:  399610 sentences:  31218
ready


/opt/conda/lib/python3.5/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full([399610, 3], 0) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/opt/conda/lib/python3.5/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full([399610, 2], 2) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/opt/conda/lib/python3.5/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(3, 0) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/opt/conda/lib/python3.5/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(2, 0) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/opt/conda/lib/python3.5/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(1, 0) will return a

399610
0 0.0


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:130: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


ValueError: Can't handle mix of multilabel-indicator and binary